In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
data = {}

In [3]:
domain = 'https://ru.wikipedia.org'
link = '/wiki/%D0%90%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%BE-%D1%82%D0%B5%D1%80%D1%80%D0%B8%D1%82%D0%BE%D1%80%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%9D%D0%BE%D0%B2%D0%BE%D1%81%D0%B8%D0%B1%D0%B8%D1%80%D1%81%D0%BA%D0%BE%D0%B9_%D0%BE%D0%B1%D0%BB%D0%B0%D1%81%D1%82%D0%B8#%D0%9A%D0%BE%D1%87%D0%B5%D0%BD%D1%91%D0%B2%D1%81%D0%BA%D0%B8%D0%B9_%D1%80%D0%B0%D0%B9%D0%BE%D0%BD'
page = requests.get(domain + link)
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
trs = soup.find('table', {'class':'standard sortable'}).find_all('tr')[6:]
trs.pop(1)
trs.pop(2)
trs.pop(-12)

for tr in trs:
    
    page_of_district_center = requests.get(domain + tr.find_all('a')[-1].get('href'))
    soup_of_district_center = BeautifulSoup(page_of_district_center.text, 'html.parser')
    
    district_center_name = tr.find_all('a')[-1].get('title').split(' ')[0]
    population = int(tr.find_all('td')[-3].text[1:-3].replace('\xa0', ''))
    if district_center_name == 'Обь':
        population += 1625631
    district_center_coordinates = soup_of_district_center.find('a', {'class': 'mw-kartographer-maplink'})
    data[district_center_name] = [district_center_coordinates.get('data-lon'), district_center_coordinates.get('data-lat'), population]

In [5]:
name_list = []
x_list = []
y_list = []
population_size = []
for key in data.keys():
    name_list.append(key)
    dt = data[key]
    x_list.append(dt[0])
    y_list.append(dt[1])
    population_size.append(dt[2])
data_view = {'Районный центр': name_list,
            'Долгота': x_list,
            'Широта': y_list,
            'Население': population_size}
df = pd.DataFrame(data_view)

In [6]:
df.head(-1)

,Районный центр,Долгота,Широта,Население
0,Обь,82.7125,54.99167,1655432
1,Баган,77.66667,54.1,15280
2,Болотное,84.4,55.66667,27194
3,Венгерово,76.74917,55.68333,18621
4,Довольное,79.67361,54.49167,15835
5,Здвинск,78.66667,54.7,14037
6,Искитим,83.3,54.63333,59836
7,Карасук,78.03333,53.73333,42882
8,Каргат,80.28333,55.2,15847
9,Колывань,82.73333,55.3,23845


In [7]:
with open("coordinates_of_district_center.json", "w") as write_file:
    json.dump(data, write_file)